# Create a Local Docker Image
In this section, we will create an IoT Edge module, a Docker container image with an HTTP web server that has a scoring REST endpoint.

## Get Global Variables

In [ ]:
import sys
sys.path.append('../../common')
from env_variables import *

## Create Web Application & Inference Server for Our ML Solution

1. To change the inference model that will be used in this sample, change the variable `IS_MODEL_NAME` based on your preferred model. Note that the model must be downloaded, as instructed in the [previous section](create_openvino_inference_engine.ipynb). The default model is         "person-vehicle-bike-detection-crossroad-1016".

2. Our model currently only accepts JPEG images. Yet, OpenCV supports many more [image formats](https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html), including PNG, TIFF, etc. If you wish to use other image formats, change the following lines of code:

    ```
    if request.headers['Content-Type'] != 'image/jpeg':
        logger.info("[AI EXT] Non JPEG content sent. Exiting the scoring event...")
    ```

3. The variable assignment of `IS_TARGET_DEVICE` indicates what type of hardware acceleration you would like to use on your IoT Edge device. You can choose from the following choices:
    * "CPU" for Intel CPU acceleration  
    * "MYRIAD" for Intel VPU acceleration
    * "GPU" for Intel GPU acceleration
    * "FPGA" for Intel FPGA acceleration

    Change the variable `IS_TARGET_DEVICE` as needed; the default is "CPU". 

In [ ]:
%%writefile $isSolutionPath/app.py
import os
from flask import Flask, request, Response
import cv2
import logging
import json
import io
from score import AnalyticsAPI
import numpy as np
import time
import datetime

import threading
from azure.iot.device import IoTHubModuleClient, MethodResponse

# Initial settings of AI model
IS_MODEL_NAME = "person-vehicle-bike-detection-crossroad-1016" # see AnalyticsAPI class for full list of models and other possibilities
IS_TARGET_DEVICE = "CPU"
IS_MODEL_PRECISION = "FP32"
IS_PROBABILITY_THRESHOLD = 0.5
IS_DEBUG_PATH = "/debug_path/" # Same value used in Manifest file. If you change this file, also change it in the deployment manifest.

app = Flask(__name__)
analyticsAPI = AnalyticsAPI(    modelName=IS_MODEL_NAME, 
                                modelPrecision=IS_MODEL_PRECISION, 
                                targetDev=IS_TARGET_DEVICE, 
                                probThreshold=IS_PROBABILITY_THRESHOLD)

# Wait some time to have the API - HW accelerators initialized
time.sleep(2)
logger = analyticsAPI.logger


# # Debug flag for writing out processed frames with bounding boxes etc. marked on them.
# # if set, debug data will be written in to "/debug_path" folder, which is bind to host PC's local drive.
# # Host PC local drive is specified in the Manifest file, and default value is host PC's /tmp/IS_DEBUG folder.
# # /tmp already exist in host pc but you should manually create and give access credentails to /tmp/IS_DEBUG (or any directory you want...)
# #
# # 0 : no debug output
# # 1 : output frames only if object detected
# # 2 : output all frames received
# #
# IS_DEBUG = 2

# # IoT Hub cloud to device method handles
# # https://docs.microsoft.com/en-us/python/api/azure-iot-device/azure.iot.device?view=azure-python (see IoTHubModuleClient)
# #
# def directMethodListener(client):
#     global analyticsAPI
#     global IS_DEBUG
#     while True:
#         try:
#             methodRequest = client.receive_method_request()  # blocking call
#             logger.info("[AI EXT] Method invoked: {0} - {1}".format(methodRequest.name, methodRequest.payload))

#             if methodRequest.name == "SetProbabilityThreshold":
#                 analyticsAPI.setProbabilityThreshold(float(methodRequest.payload))
#                 callResult = str(analyticsAPI.getProbabilityThreshold())

#             elif methodRequest.name == "SetModel":
#                 analyticsAPI.setModelName(str(methodRequest.payload))
#                 callResult = analyticsAPI.getModelName()

#             elif methodRequest.name == "SetModelPrecision":
#                 analyticsAPI.setModelPrecision(str(methodRequest.payload))
#                 callResult = analyticsAPI.getModelPrecision()

#             elif methodRequest.name == "SetTargetDevice":
#                 analyticsAPI.setTargetDevice(str(methodRequest.payload))
#                 callResult = analyticsAPI.getTargetDevice()

#             elif methodRequest.name == "SetDebug":
#                 IS_DEBUG = int(methodRequest.payload)
#                 callResult = str(IS_DEBUG)

#             else:
#                 continue

#             responsePayload = {"Success": "{0} - {1}".format(methodRequest.name, callResult)}
#             methodResponse = MethodResponse(methodRequest.request_id, 200, payload=responsePayload)
#             client.send_method_response(methodResponse)

#         except Exception as e:
#             responsePayload = {"Exception": "{0} - {1}".format(methodRequest.name, str(e))}
#             methodResponse = MethodResponse(methodRequest.request_id, 400, payload=responsePayload)
#             client.send_method_response(methodResponse)

# #
# # Initialize IoTHubDeviceClient to listen method calls coming from cloud.
# #
# iotHubModuleClient = IoTHubModuleClient.create_from_edge_environment(websockets=True)
# iotHubModuleClient.connect()

# # Start IoTHub event callback threads
# threadDirectMethodListener = threading.Thread(target=directMethodListener, args=(iotHubModuleClient,))
# threadDirectMethodListener.daemon = True
# threadDirectMethodListener.start()

# TODO: handle twin updates...

@app.route("/score", methods = ['POST'])
def scoreRRS():
    global analyticsAPI
    global logger
    global IS_DEBUG
    
    # uncomment for extended debug log
    # logger.info("[AI EXT] Received scoring request. Header: {0}".format(json.dumps(dict(request.headers))))
    # logger.info("[AI EXT] Received scoring request")

    try:
        # We accept only "image/jpeg" in this sample, but you can change the line below for other formats. 
        if request.headers['Content-Type'] != 'image/jpeg':
            logger.info("[AI EXT] Non JPEG content sent. Exiting the scoring event...")
            #return Response(json.dumps({}), status= 415, mimetype ='application/json')
            return Response(status= 500) #Internal Server error - not expected

        # Before going forward, return not ready, if it is the case...
        if not analyticsAPI.initialized:
            logger.info("[AI EXT] {}".format(analyticsAPI.INF_STAT_NOT_READY))
            return  Response(status= 500) #Internal Server error - not expected
            #return  Response(json.dumps({"status": analyticsAPI.INF_STAT_NOT_READY}), status= 200, mimetype ='application/json')  

        # get request as byte stream
        reqBody = request.get_data(False)

        # convert from byte stream
        inMemFile = io.BytesIO(reqBody)

        # load a sample image
        inMemFile.seek(0)
        fileBytes = np.asarray(bytearray(inMemFile.read()), dtype=np.uint8)
        cvImage = cv2.imdecode(fileBytes, cv2.IMREAD_COLOR)

        # call scoring function
        result = analyticsAPI.score(cvImage)            

        # color = (255, 0, 0) # in BGR order

        # if IS_DEBUG > 0:
        #     jsonResult = json.loads(result)
        #     statusCode = int(jsonResult["status"])
        #     objectCount = 0
            
            # if statusCode == 0:
            #     objectCount = int(jsonResult["object_count"])
            #     if objectCount > 0:
            #         jsonResult = jsonResult["result"]
            #         for k in jsonResult:
            #             detRes = jsonResult[k]
            #             xmin = int(detRes["xmin"])
            #             ymin = int(detRes["ymin"])
            #             xmax = int(detRes["xmax"])
            #             ymax = int(detRes["ymax"])
            #             classId = str(detRes["label"])
            #             confidence = str(detRes["confidence"])

            #             cv2.rectangle(cvImage, (xmin, ymin), (xmax, ymax), color, 2)
            #             cv2.putText(cvImage, classId + " - " + confidence, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 1, color, 1)

            # if (IS_DEBUG > 1) or (objectCount > 0):
            #     camID = request.args.get('cam', default = 0, type = int)
            #     outPath = os.path.join(IS_DEBUG_PATH, "cam{0:02d}".format(camID))
            #     os.makedirs(outPath, exist_ok=True)
            #     outFileName = os.path.join(outPath, str(datetime.datetime.now()).replace(" ","_") + ".jpg")

            #     # resizedImg = cv2.resize(cvImage, (800, 600), interpolation = cv2.INTER_AREA)
            #     # retval = cv2.imwrite(outFileName, resizedImg)
            #     retval = cv2.imwrite(outFileName, cvImage)

            # # Log only in debug mode
            # logger.info("[AI EXT] Sending response. Status: {0} - Count: {1}".format(statusCode, objectCount))

        logger.info("[AI EXT] Sending response.")
        return Response(result, status= 200, mimetype ='application/json')

    except Exception as e:
        logger.info("[AI EXT] Exception (scoreRRS): {0}".format(str(e)))
        return Response(response='Exception occured while processing the image.', status=500)   
        #return Response(json.dumps({}), status= 200, mimetype ='application/json')   
    
@app.route("/")
def healthy():
    return "Healthy"

# Version
@app.route('/version', methods = ['GET'])
def version_request():
    global analyticsAPI
    return analyticsAPI.version()

# About
@app.route('/about', methods = ['GET'])
def about_request():
    global analyticsAPI
    return analyticsAPI.about()

if __name__ == "__main__":
    while not analyticsAPI.initialized:
        logger.info("[AI EXT] Waiting AI module to be initialized. (app.py)")
        time.sleep(1)
        
    app.run(host='0.0.0.0', port=5444)

In the cell above, 5444 is the internal port of the webserver app that listens the requests. Next, we will map it to different ports to expose it externally.

In [ ]:
%%writefile $isSolutionPath/wsgi.py
from app import app as application

def create():
    application.run(host='127.0.0.1', port=5444)

In [ ]:
import os
os.makedirs(os.path.join(isSolutionPath, "nginx"), exist_ok=True)

The exposed port of the web app is now 5001, while the internal one is still 5444.

In [ ]:
%%writefile $isSolutionPath/nginx/app
server {
    listen 5001;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5444;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

In [ ]:
%%writefile $isSolutionPath/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

In [ ]:
%%writefile $isSolutionPath/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('[AI EXT] READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('[AI EXT] Terminating supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('[AI EXT] Could not terminate supervisor: ' + e.strerror + '\n')
            write_stdout('[AI EXT] RESULT 2\nOK')

main()

In [ ]:
import os
os.makedirs(os.path.join(isSolutionPath, "etc"), exist_ok=True)

In [ ]:
%%writefile $isSolutionPath/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

# TODO DO WE NEED THIS?
environment=LD_LIBRARY_PATH=%(ENV_LD_LIBRARY_PATH)s,INTEL_CVSDK_DIR=%(ENV_INTEL_CVSDK_DIR)s,OpenCV_DIR=%(ENV_OpenCV_DIR)s,InferenceEngine_DIR=%(ENV_InferenceEngine_DIR)s,PYTHONPATH=%(ENV_PYTHONPATH)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s,HDDL_INSTALL_DIR=%(ENV_HDDL_INSTALL_DIR)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/isserver
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/isserver
events=PROCESS_STATE_FATAL
priority=2

In [ ]:
# TODO Do we need this?

%%writefile $isSolutionPath/requirements.txt
pillow<7.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

## Create a Docker File to Containerize the ML Solution and Web App Server

> <span style="color:red; font-weight: bold; font-size:1.1em;"> [!IMPORTANT] </span>  

> The OpenVINO™ Toolkit is a licenced software. To ensure that you are using the latest version of the OpenVINO™ Toolkit, follow these instructions to obtain a licensed download link:  

> 1) Go to the [Intel donwload link](https://software.intel.com/en-us/openvino-toolkit/choose-download/free-download-linux) for the OpenVINO™ Toolkit

> 2) Click on the "Register & Download" button  

> <img src="../../../../../images/_openvino_img_03_001.jpg" width=400 alt="> Figure: Register & Download."/>  

> 3) Fill in the form and click submit 

> <img src="../../../../../images/_openvino_img_03_002.jpg" width=400 alt="> Figure: Fill the form."/>  

> 4) Over the "Full Package" link, right click and get the link which should look like something:  
    http://registrationcenter-download.intel.com/akdlm/irc_nas/<SOMECODE\>/l_openvino_toolkit_p_2020.3.194.tgz  
    
> <img src="../../../../../images/_openvino_img_03_003.jpg" width=400 alt="> Figure: Download link."/>  

> 5) In the below cell, set the value of variable "openVinoToolkitDownloadLink" to the download link you have.

In [ ]:
# As described above, set the value of variable "openVinoToolkitDownloadLink" to the download link you have (below is sample URI, just remove it and use your own address)

# openVinoToolkitDownloadLink = "http://registrationcenter-download.intel.com/akdlm/irc_nas/<SOMECODE>/l_openvino_toolkit_p_2020.3.194.tgz"
openVinoToolkitDownloadLink = "<YOUR_LINK>"

In [ ]:
%%writefile $isSolutionPath/Dockerfile

FROM ubuntu:18.04

USER root

ARG WORK_DIR=/isserver
ENV WORK_DIR ${WORK_DIR}
ENV PATH /opt/miniconda/bin:${PATH}

RUN mkdir -p ${WORK_DIR}

WORKDIR ${WORK_DIR}

#
# Install base
#
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        wget \
        locales \
        # Python environment
        python3 \
        python3-setuptools &&\
    #
    # Dependencies: conda
    wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh --no-check-certificate &&\ 
    /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/miniconda &&\
    #
    # Cleaning
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Install OpenVINO™
#
#COPY l_openvino_toolkit_p_2020.1.023.tgz ${WORK_DIR}
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        cpio \
        udev \
        unzip \
        autoconf \
        automake \
        libtool

RUN wget --quiet IS_OPENVINO_TOOLKIT_DOWNLOAD_LINK -O ${WORK_DIR}/l_openvino_toolkit_p_2020.1.023.tgz &&\
    pattern="COMPONENTS=DEFAULTS" &&\
    replacement="COMPONENTS=intel-openvino-ie-sdk-ubuntu-bionic__x86_64;intel-openvino-ie-rt-cpu-ubuntu-bionic__x86_64;intel-openvino-ie-rt-vpu-ubuntu-bionic__x86_64;intel-openvino-opencv-lib-ubuntu-bionic__x86_64" &&\
    tar -xzf l_openvino_toolkit*.tgz &&\
    cd l_openvino_toolkit* &&\
    sed -i "s/$pattern/$replacement/" silent.cfg &&\
    sed -i "s/decline/accept/g" silent.cfg &&\
    /bin/bash ./install.sh -s silent.cfg &&\
    cd - &&\
    cd /opt/intel/openvino/install_dependencies &&\
    /bin/bash ./install_openvino_dependencies.sh &&\
    # setup environment variables
    echo "source /opt/intel/openvino/bin/setupvars.sh" >> /root/.bashrc &&\
    #
    # Cleaning
    cd ${WORK_DIR} &&\
    rm -rf * &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Set environment variables as in ${INTEL_OPENVINO_DIR}/bin/setupvars.sh
ENV INTEL_OPENVINO_DIR /opt/intel/openvino
ENV LD_LIBRARY_PATH ${INTEL_OPENVINO_DIR}/opencv/lib:${INTEL_OPENVINO_DIR}/deployment_tools/ngraph/lib:/opt/intel/opencl:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/gna/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/mkltiny_lnx/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/tbb/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/lib/intel64
ENV INTEL_CVSDK_DIR ${INTEL_OPENVINO_DIR}
ENV OpenCV_DIR ${INTEL_OPENVINO_DIR}/opencv/cmake
ENV InferenceEngine_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/share
ENV PYTHONPATH ${INTEL_OPENVINO_DIR}/python/python3.7:${INTEL_OPENVINO_DIR}/python/python3:${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/accuracy_checker:${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer
ENV PATH ${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:${PATH}
ENV HDDL_INSTALL_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl

#
# Exclude UDEV by rebuilding libusb without UDEV support
RUN cp ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/97-myriad-usbboot.rules /etc/udev/rules.d/ &&\
    ldconfig &&\
    cd /opt && wget --quiet --no-check-certificate http://github.com/libusb/libusb/archive/v1.0.22.zip -O /opt/v1.0.22.zip &&\
    unzip v1.0.22.zip && cd libusb-1.0.22 &&\
    ./bootstrap.sh &&\
    ./configure --disable-udev --enable-shared &&\
    make -j4

RUN apt-get update &&\
    apt-get install -y --no-install-recommends libusb-1.0-0-dev &&\
    cd /opt &&\
    rm -rf /var/lib/apt/lists/* &&\
    cd /opt/libusb-1.0.22/libusb &&\
    /bin/mkdir -p '/usr/local/lib' &&\
    /bin/bash ../libtool --mode=install /usr/bin/install -c libusb-1.0.la '/usr/local/lib' &&\
    /bin/mkdir -p '/usr/local/include/libusb-1.0' &&\
    /usr/bin/install -c -m 644 libusb.h '/usr/local/include/libusb-1.0' &&\
    /bin/mkdir -p '/usr/local/lib/pkgconfig' &&\
    cd /opt/libusb-1.0.22/ &&\
    /usr/bin/install -c -m 644 libusb-1.0.pc '/usr/local/lib/pkgconfig' &&\
    ldconfig

#
# Install ML solution
RUN apt-get update && apt-get install -y --no-install-recommends \
    nginx \
    supervisor &&\
    pip install \
        numpy \
        azure-iot-device
        
ADD . ${WORK_DIR}
ADD etc /etc

RUN rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default &&\
    cp ${WORK_DIR}/nginx/app /etc/nginx/sites-available/ &&\
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&\
    pip install -r ${WORK_DIR}/requirements.txt &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

EXPOSE 5001
CMD ["supervisord", "-c", "/isserver/etc/supervisord.conf"]

In [ ]:
# Update Docker file with custom environment variable: IoT Edge device's connection string
filePath = isSolutionPath+"/Dockerfile"
file = open(filePath)
dockerFileTemplate = file.read()
dockerFileTemplate = dockerFileTemplate.replace("IS_OPENVINO_TOOLKIT_DOWNLOAD_LINK", "\""+openVinoToolkitDownloadLink+"\"")

with open(filePath, 'wt', encoding='utf-8') as outputFile:
    outputFile.write(dockerFileTemplate)

## Create a Local Docker Image
Finally, we will create a Docker image locally. We will later host the image in a container registry like Docker Hub, Azure Container Registry, or a local registry.

To run the following code snippet, you must have the pre-requisities mentioned in [the requirements page](/yolov3-ngpu-onnx/01_requirements.md). Most notably, we are running the `docker` command without `sudo`.

> <span>[!WARNING]</span>
> Please ensure that Docker is running before executing the cell below. Execution of the cell below may take several minutes. 

In [ ]:
!docker build -t $containerImageName --file ./$isSolutionPath/Dockerfile ./$isSolutionPath

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   